In [ ]:
pip install yfinance newsapi-python pandas

In [ ]:
import os
import yfinance as yf
from newsapi import NewsApiClient
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()  # loads from .env file
api_key = os.getenv("NEWS_API")

class CollectorAgent:
    def __init__(self, news_api_key):
        self.news_api = NewsApiClient(api_key=news_api_key)
    
    def get_stock_data(self, ticker, period="1mo"):
        """Fetches historical stock data."""
        print(f"Agent collecting stock data for {ticker}...")
        stock = yf.Ticker(ticker)
        df = stock.history(period=period)
        return df[['Close', 'Volume']]

    def get_financial_news(self, query, days_back=2):
        """Fetches recent news headlines."""
        print(f"Agent collecting news for '{query}'...")
        
        # Calculate date range
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back)
        
        articles = self.news_api.get_everything(
            q=query,
            from_param=start_date.strftime('%Y-%m-%d'),
            to=end_date.strftime('%Y-%m-%d'),
            language='en',
            sort_by='relevancy',
            page_size=10
        )
        
        news_data = []
        for article in articles['articles']:
            news_data.append({
                'date': article['publishedAt'][:10],
                'title': article['title'],
                'source': article['source']['name']
            })
            
        return pd.DataFrame(news_data)

# --- EXECUTION ---
# Replace 'YOUR_API_KEY' with your actual NewsAPI key
api_key = api_key 
agent = CollectorAgent(api_key)

# 1. Get Apple (AAPL) stock data
stock_df = agent.get_stock_data("AAPL", period="5d")
print(f"\nLast 5 days stock data:\n{stock_df.head()}")

# 2. Get news about Apple
news_df = agent.get_financial_news("Apple Inc")
print(f"\nRecent Headlines:\n{news_df[['date', 'title']].head()}")